In [0]:
DEBUG = False
FIELD = 'ursaminor'
OBS_FILE = f'$PFS_TARGETING_DATA/data/targeting/dSph/{FIELD}/{FIELD}_tpall3e_g24.cat'
CONFIG_FILES = [
    '/home/dobos/project/ga_targeting/configs/netflow/SSP/dSph/_common.py',
    f'/home/dobos/project/ga_targeting/configs/netflow/SSP/dSph/{FIELD}.py' 
]
ASSIGNMENTS_FILE = ''

# Observation time frame in HST
TIME_FROM = '2025-04-01 12:00:00'
TIME_TO = '2025-04-02 12:00:00'

In [0]:
import os, sys
from datetime import datetime
import dateutil.parser
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
from matplotlib.colors import LinearSegmentedColormap
from matplotlib.patches import Ellipse, Circle
from matplotlib.gridspec import GridSpec

In [0]:
plt.rc('font', size=6)

In [0]:
%load_ext autoreload
%autoreload 2

In [0]:
if DEBUG and 'debug' not in globals():
    import debugpy
    debugpy.listen(('0.0.0.0', 5698))
    debug = True

# Plot definitions

In [0]:
from astropy.time import Time, TimeDelta
from astropy.coordinates import SkyCoord, EarthLocation, AltAz, get_body
from astroplan import Observer
from astroplan.moon import moon_phase_angle, moon_illumination
import astropy.units as u

from pfs.utils.coordinates import Subaru_POPT2_PFS

from pfs.ga.targeting.targets.dsph import GALAXIES as DSPH_FIELDS
from pfs.ga.targeting.targets.m31 import M31_FIELDS
from pfs.ga.targeting.instrument import *
from pfs.ga.targeting.diagram import CMD, CCD, FOV, FP, ColorAxis, MagnitudeAxis
from pfs.ga.targeting.photometry import Photometry, Magnitude, Color
from pfs.ga.targeting.projection import Pointing, WcsProjection
from pfs.ga.targeting.config.netflow import NetflowConfig

In [0]:
field = DSPH_FIELDS[FIELD]
hsc = field.get_photometry()
cmd = field.get_cmd()
ccd = field.get_ccd()

In [0]:
pointing = field.get_center()
pointing

In [0]:
wcs = WcsProjection(pointing, proj='TAN')
fov = FOV(projection=wcs)

# Load the config files

In [0]:
config = field.get_netflow_config()
config.load(CONFIG_FILES, ignore_collisions=True)

In [0]:
config.pointings

In [0]:
config.instrument_options

# Load observations

In [0]:
from pfs.ga.targeting.io import ObservationSerializer

In [0]:
obs = field.get_text_observation_reader().read(os.path.expandvars(OBS_FILE))
obs.data.shape

# Calculate the whitening matrix

This basically determines the second moments of the coordinate distribution
and yield the axes of the ellipsoid and the position angle

In [0]:
from pfs.ga.targeting.projection import Whitening
from pfs.ga.targeting.radialprofile import King

In [0]:
# Iteratively determine a whitening matrix that converts the
# spherical coordinates into deprojected coordinates

w = Whitening(projection=wcs)

# Ursa Minor
w_mask = w.create(obs, iterations=30, s_cut=2.2)

# Draco
# w_mask = w.create(obs, iterations=20, s_cut=2.2)

w.M, w.reverse(w.M), field.pos

In [0]:
[dsph_maj, dsph_min] = w.S
dsph_pa = np.degrees(np.angle(w.Vh[0, 0] + 1j * w.Vh[1, 0]))
dsph_maj, dsph_min, dsph_pa

In [0]:
f, ax = plt.subplots(1, 1, figsize=(8, 8), dpi=240)

(wx, wy) = w.apply(obs.get_coords())
ax.plot(wx, wy, '.', ms=0.1)
ax.plot(wx[w_mask], wy[w_mask], '.', ms=0.1)
ax.set_aspect('equal')
ax.grid()

In [0]:
w.M, w.W, w.reverse(w.M)

In [0]:
w.M, w.reverse(w.M), field.pos

In [0]:
# Fit profile to all stars within the color cut

# Ursa Minor
king = King(transformation=w, R_max=9, bins=30)

# Draco
# king = King(transformation=w, R_max=10, bins=30)

king.histogram(obs) #, mask=w_mask)
[S_b, S_0, R_c, R_t], pcov = king.fit()

S_b, S_0, R_c, R_t

In [0]:
# Verify the King profile fit

f, ax = plt.subplots(1, 1, figsize=(3.4, 2.5), dpi=120)

R, log_S, log_S_sigma = king.get_log_S()

ax.errorbar(R, log_S, 3 * log_S_sigma)
ax.set_xlabel('deprojected R')
ax.set_ylabel(r'$\log \Sigma$')

R = np.linspace(0, 11, 200)
log_S = king.log_eval(R)
ax.plot(R, log_S)

# Define pointings

In [0]:
from astropy.coordinates import SkyCoord
from astropy.time import Time
from astropy.coordinates import CartesianRepresentation
from astropy.coordinates.matrix_utilities import rotation_matrix
import astropy.units as u

from pfs.ga.targeting.projection import Pointing

In [0]:
# Cartesian coordinate system
SkyCoord(0, 90, unit='deg').cartesian       # (0, 0, 1)     -> z axis points to north
SkyCoord(0, 0, unit='deg').cartesian         # (1, 0, 0)     -> x axis points to us
SkyCoord(90, 0, unit='deg').cartesian        # (0, 1, 0)     -> y axis points to east

In [0]:
def get_full_rotation(sep, pa, ra, dec):
    r1 = rotation_matrix(sep * u.deg, 'z')  # separation of the pointing center from the center of the object
    r2 = rotation_matrix(pa * u.deg, 'x')   # position angle of the ellipse
    r3 = rotation_matrix(dec * u.deg, 'y')    # declination
    r4 = rotation_matrix(-ra * u.deg, 'z')    # right ascension

    return r4 @ r3 @ r2 @ r1

def calculate_center(sep, pa, ra, dec):
    r = get_full_rotation(sep, pa, ra, dec)
    c = SkyCoord(CartesianRepresentation(1, 0, 0).transform(r))
    return c.ra.value, c.dec.value

# print(r1, r2, r3, r4, r4 @ r3 @ r2 @ r1)

# print(SkyCoord(CartesianRepresentation(1, 0, 0)))
# print(SkyCoord(CartesianRepresentation(1, 0, 0).transform(r1)))
# print(SkyCoord(CartesianRepresentation(1, 0, 0).transform(r2 @ r1)))
# print(SkyCoord(CartesianRepresentation(1, 0, 0).transform(r3 @ r2 @ r1)))
# print(SkyCoord(CartesianRepresentation(1, 0, 0).transform(r4 @ r3 @ r2 @ r1)))

In [0]:
field.pos.ra.value, field.pos.dec.value, calculate_center(1.5, 40, field.pos.ra.value, field.pos.dec.value)

In [0]:
dsph_pa

In [0]:
center = field.get_center()
obs_time = Time('2025-03-25T10:00:00')        # midnight in Hawaii

# Use the field defaults
pointings = field.get_pointings(SubaruPFI)

# Use the config file
# pointings = [ p.get_pointing(obs_time=obs_time) for p in config.pointings ]

# Generate pointings in a pattern
# pa = 0 # 180 + dsph_pa
# pa = 0
# pointings = [
#     Pointing(*calculate_center(0.47, 0, field.pos.ra.value, field.pos.dec.value), posang=30, obs_time=obs_time),
#     Pointing(*calculate_center(-0.47, 0, field.pos.ra.value, field.pos.dec.value), posang=30, obs_time=obs_time),

#     Pointing(*calculate_center(0.25, 90, field.pos.ra.value, field.pos.dec.value), posang=0, obs_time=obs_time),
#     Pointing(*calculate_center(-0.25, 90, field.pos.ra.value, field.pos.dec.value), posang=0, obs_time=obs_time),
# ]

center, pointings

# Plot the pointings

In [0]:
f, ax = plt.subplots(1, 1, figsize=(8, 8), dpi=240, subplot_kw=dict(projection=wcs.wcs))

fov.plot_observation(ax, obs, c='lightgrey', size=0.5)

# Plot the tidal radius from the King profile fit
ell = king.get_ellipse(R_t)
fov.plot(ax, ell, native_frame='world', ls='--', lw=1, c='k')
ell = king.get_ellipse(R_c)
fov.plot(ax, ell, native_frame='world', ls='-', lw=1, c='k')

# Plot the poiting center
# fov.plot(ax, [field.pos.ra.value], [field.pos.dec.value], fmt='+r', ms=10, native_frame='world')

for pp in pointings:
    wfc = SubaruWFC(pp)
    pfi = SubaruPFI(wfc, instrument_options=config.instrument_options)

    pfi.plot_focal_plane(ax, fov, corners=True, c='r', ls='-')
    pfi.plot_focal_plane(ax, fov, fill=True, alpha=0.2)

ax.set_xlim(1, -3)
ax.set_ylim(-3, 1)
ax.grid()

# Calculate and plot the elevation and rotator angle

In [0]:
obs = Observer.at_site("Subaru", timezone="US/Hawaii")
hawaii_tz = TimeDelta(-10 * u.hr)

In [0]:
config.pointings

In [0]:
time_from = Time(dateutil.parser.parse(TIME_FROM)) - hawaii_tz
time_to = Time(dateutil.parser.parse(TIME_TO)) - hawaii_tz

time_from, time_to

In [0]:
date_from = np.floor(time_from.jd - 0.5)
date_to = np.ceil(time_to.jd + 0.5)

dates = Time(np.linspace(date_from, date_to + 1, int(date_to - date_from + 2)), format='jd')
dates.to_datetime()

In [0]:
sunrise = obs.sun_rise_time(dates, which='previous') + hawaii_tz
sunset = obs.sun_set_time(dates, which='next') + hawaii_tz

# HST
sunrise.to_datetime(), sunset.to_datetime()

In [0]:
def radecpa_to_altazinr(pos, pa, obs_time):
    altaz = pos.transform_to(AltAz(obstime=obs_time, location=Subaru_POPT2_PFS.Lsbr))
    az = altaz.az.deg
    alt = altaz.alt.deg

    # Instrument rotator angle
    subaru = Subaru_POPT2_PFS.Subaru()
    paa = subaru.radec2inr(pos.ra, pos.dec, obs_time)
    inr = (paa + pa + 180) % 360. - 180.
    # inr = paa + pa

    return alt, az, inr

In [0]:
obs_time = Time(np.linspace(date_from, date_to, 300), format='jd')
obs_time[0], obs_time[-1]

In [0]:
obs_time[0], obs_time[-1]

In [0]:
f, ax = plt.subplots(1, 1, figsize=(7, 5), dpi=240)
ax2 = ax.twinx()

# matplotlib like python datetime
xx = (obs_time + hawaii_tz).to_datetime()

# Shade areas in black when the Sun is below the horizon
for t1, t2 in zip(sunset[:-2].to_datetime(), sunrise[2:].to_datetime()):
    ax.axvspan(t1, t2, color='black', alpha=1)

# Plot the rotator angle for each pointing
for p in config.pointings:
    p = p.get_pointing()
    alt, az, inr = radecpa_to_altazinr(p.pos, p.posang, obs_time=(obs_time))
    # alt, az, inr = radecpa_to_altazinr(p.pos, 0, obs_time=(obs_time))

    inr_mask = np.abs(inr) > 178
    inr[inr_mask] = np.nan

    # Instrument rotator angle
    ax2.plot(xx, inr, c='orange')

# Plot the elevation of the Moon
moon = get_body('Moon', obs_time, Subaru_POPT2_PFS.Lsbr)
moon_phase = moon_illumination(obs_time)
altaz = moon.transform_to(AltAz(obstime=obs_time, location=Subaru_POPT2_PFS.Lsbr))
ax.plot(xx, altaz.alt.degree, '--y')

# Plot target elevation
p = field.get_center_pointing(include_motion=False)
alt, az, inr = radecpa_to_altazinr(p.pos, 0, obs_time=(obs_time))
ax.plot(xx, alt, '-c')

ax.set_xlim((time_from + hawaii_tz).to_datetime(), (time_to + hawaii_tz).to_datetime())

ax.set_yticks(np.linspace(0, 90, 7))
ax.set_ylim(0, 90)

ax.tick_params(axis='x', labelrotation=90)

ax.set_xlabel('time [HST]')
ax.set_ylabel('elevation')
ax.set_title(f'Field: {FIELD}, moon phase: {moon_phase.mean():0.2f}')

ax.grid(True)

ax2.set_yticks(np.linspace(-180, 180, 7))
ax2.set_ylim(-180, 180)

ax2.set_ylabel('rotator angle')